In [35]:
#!/usr/bin/env python
from pathlib import Path

import trio # type: ignore

from lean_client.trio_server import TrioLeanServer

async def main():
    lines = Path('test.lean').read_text().split('\n')

    async with trio.open_nursery() as nursery:
        server = TrioLeanServer(nursery, debug=False)
        await server.start()
        test = await server.full_sync('test.lean')

        for i, line in enumerate(lines):
            before = await server.state('test.lean', i+1, 0)
            after = await server.state('test.lean', i+1, len(line))
            if before or after:
                print(f'Line {i+1}: {line}')
                print(f'State before:\n{before}\n')
                print(f'State after:\n{after}\n')

        server.kill()
        nursery.cancel_scope.cancel()

if __name__ == '__main__':
    trio.run(main)

/home/clousty8/.local/share/virtualenvs/test_lean/lib/python3.10/site-packages/lean_client/trio_server.py:41: TrioDeprecationWarning: trio.open_process is deprecated since Trio 0.20.0; use trio.lowlevel.open_process instead (https://github.com/python-trio/trio/issues/1104)
  self.process = await trio.open_process(


In [ ]:
import trio

from lean_client.trio_server import TrioLeanServer

async def check_theorem():
    async with trio.open_nursery() as nursery:
            # Initialize the Lean server
        lean_server = TrioLeanServer(nursery, debug=False)
        await lean_server.start()

        # Define the Lean code to check the theorem
        lean_code = """
theorem my_theorem : 2 + 1 = 4 :=
    begin
        refl
    end
        """

        # Create a Lean file and fully compile it
        filename = "my_theorem.lean"
        await lean_server.full_sync(filename, lean_code)

        # Check the tactic state of the theorem
        line = 1  # Adjust the line number based on your Lean code
        col = len(lean_code)+1  # Adjust the column number based on your Lean code
        tactic_state = await lean_server.state(filename, line, col)
        print(tactic_state)

        # Check if the theorem is provable
        if "no goals" in tactic_state:
            print("The theorem is provable: 2 + 2 = 4")
        else:
            print("The theorem is not provable.")

        # Clean up and kill the Lean server
        lean_server.kill()
        nursery.cancel_scope.cancel()


if __name__ == "__main__":
    trio.run(check_theorem)


/home/clousty8/.local/share/virtualenvs/test_lean/lib/python3.10/site-packages/lean_client/trio_server.py:41: TrioDeprecationWarning: trio.open_process is deprecated since Trio 0.20.0; use trio.lowlevel.open_process instead (https://github.com/python-trio/trio/issues/1104)
  self.process = await trio.open_process(
